In [1]:
import os
import dataclasses
import hydra
from hydra.core.config_store import ConfigStore
from omegaconf import DictConfig, OmegaConf

In [2]:
from rlprompt.trainers import TrainerConfig, make_trainer
from rlprompt.modules import SQLModuleConfig, make_sql_module
from rlprompt.models import (LMAdaptorModelConfig, SinglePromptModelConfig,
                             make_lm_adaptor_model, make_single_prompt_model)
from rlprompt.utils.utils import (colorful_print, compose_hydra_config_store,
                                  get_hydra_output_dir)
from tst_helpers import (PromptedTextStyleTransferRewardConfig,
                         TextStyleTransferDatasetConfig,
                         make_prompted_text_style_transfer_reward,
                         make_text_style_transfer_datasets,
                         get_style_classifier)


### Set default configuration

In [3]:
config_list = [PromptedTextStyleTransferRewardConfig,
                TextStyleTransferDatasetConfig, LMAdaptorModelConfig,
                SinglePromptModelConfig, SQLModuleConfig, TrainerConfig]
cs = compose_hydra_config_store('base_tst', config_list)


In [4]:
config=OmegaConf.load('rayabharam.yaml')
assert type(config) == DictConfig

In [5]:
colorful_print(OmegaConf.to_yaml(config), fg='red')

task_lm: distilgpt2
task_top_k: 10
style_classifier: ???
style_tokenizer: bert-base-uncased
style_batch_size: 32
pad_token: <|endoftext|>
num_repeats: 4
num_samples: 32
num_bootstraps: 4
compute_zscore: true
lower_outputs: true
control_output_length: true
template: '{prompt} "{sentence_1}" "'
end_punct: '"'
dataset: yelp
dataset_seed: null
direction: 0_to_1
base_path: ./data
max_size: null
max_length: null
max_length_tokenizer: null
policy_lm: distilgpt2
hidden_size: 2048
logit_bias: -10.0
fluent: false
fluent_top_k: 20
max_decoding_length: 5
eos_token_id: null
prompt_length: 5
prompt_train_batch_size: 8
prompt_infer_batch_size: 16
source_str: <|endoftext|>
sql_loss_impl: v2_v2r_v3_v3r
training_mode: sql-onpolicy
mix_strategy: null
target_update_method: polyak
target_update_steps: null
target_learning_rate: 0.001
reward_shaping: true
reward_shaping_old_min: 0.0
reward_shaping_old_max: 1.0
reward_shaping_new_min: -20.0
reward_shaping_new_max: 80.0
top_k: 50
top_p: 1.0
num_beams: 1
train

In [6]:
train_dataset, val_dataset, test_dataset = make_text_style_transfer_datasets(config)
print('Train Size:', len(train_dataset))
print('Examples:', train_dataset[:5])
print('Val Size', len(val_dataset))
print('Examples:', val_dataset[:5])

output_dir="outputs/2023-05-04/18-25-40"

Train Size: 176787
Examples: {'source_texts': ['i was sadly mistaken.', 'so on to the hoagies, the italian is general run of the mill.', 'minimal meat and a ton of shredded lettuce.', 'nothing really special & not worthy of the $_num_ price tag.', 'second, the steak hoagie, it is atrocious.'], 'target_labels': ['LABEL_1', 'LABEL_1', 'LABEL_1', 'LABEL_1', 'LABEL_1']}
Val Size 16
Examples: {'source_texts': ['ok never going back to this place again.', 'easter day nothing open, heard about this place figured it would ok.', 'the host that walked us to the table and left without a word.', 'it just gets worse.', 'the food tasted awful.'], 'target_labels': ['LABEL_1', 'LABEL_1', 'LABEL_1', 'LABEL_1', 'LABEL_1']}


In [11]:
policy_model = make_lm_adaptor_model(config)
prompt_model = make_single_prompt_model(policy_model, config)
config.style_classifier = get_style_classifier('train', config)
config.max_train_steps=1000
reward = make_prompted_text_style_transfer_reward(config)
algo_module = make_sql_module(prompt_model, reward, config)

config.save_dir = os.path.join(output_dir, config.save_dir)
trainer = make_trainer(algo_module, train_dataset, val_dataset, config)

Task LM: distilgpt2


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Train model

In [12]:
trainer.train(config=config)

SQL_ON/loss,▁█
SQL_ON/loss-normalized,▁█
SQL_ON/rewards/mean_content,█▁
SQL_ON/rewards/mean_reward,▁█
SQL_ON/rewards/mean_style,▁█
SQL_ON/rewards/num_tokens_explored,▁█
SQL_ON/rewards/raw,▁█
SQL_ON/rewards/shaped,▁█
SQL_ON/rewards/sum_reward,▁█
SQL_ON/rewards/top_content,▁█
SQL_ON/rewards/top_style,█▁


/home/nqr5356/tech/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


1 | ['Bitcoin', 'Login', 'Microsoft', 'ĠBy', 'Sword'] | BitcoinLoginMicrosoft BySword | i was sadly mistaken. | i's a very nice guy. | Top Content: 33.73 | Top Style: 99.89 | Top Reward: 66.81 | Reward: 61.71
1 | ['Square', 'Guide', 'Senate', 'Message', 'Info'] | SquareGuideSenateMessageInfo | i was sadly mistaken. | i have a few friends who are also friends. | Top Content: 25.83 | Top Style: 99.42 | Top Reward: 62.62 | Reward: 53.73
1 | ['Story', 'Menu', 'Login', 'Shadow', 'Square'] | StoryMenuLoginShadowSquare | i was sadly mistaken. | it was a very good idea, | Top Content: 37.22 | Top Style: 99.88 | Top Reward: 68.55 | Reward: 63.49
1 | ['ĠNEW', 'Description', 'Spider', 'Effect', 'Square'] |  NEWDescriptionSpiderEffectSquare | i was sadly mistaken. | i'm very happy to report my mistake. | Top Content: 33.59 | Top Style: 97.04 | Top Reward: 65.31 | Reward: 62.54
1 | ['Quantity', 'President', 'Server', 'President', 'Gender'] | QuantityPresidentServerPresidentGender | so on to the hoa

/home/nqr5356/tech/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2 | ['Depth', 'Login', 'Letter', 'Background', 'Login'] | DepthLoginLetterBackgroundLogin | minimal meat and a ton of shredded lettuce. | this is the recipe for a great breakfast with bacon, cheese and a bunch of other meats. | Top Content: 23.19 | Top Style: 99.87 | Top Reward: 61.53 | Reward: 58.93
2 | ['Numbers', 'Background', 'Numbers', 'Synopsis', 'Integer'] | NumbersBackgroundNumbersSynopsisInteger | minimal meat and a ton of shredded lettuce. | a ton of sliced red onions with some fresh ground pepper. | Top Content: 31.41 | Top Style: 99.96 | Top Reward: 65.69 | Reward: 63.77


...

988 | ['Rated', 'Pros', 'Rated', 'Ġexcellence', 'ĠGreat'] | RatedProsRated excellence Great | neither one greeted me or asked me what i would have like to order. | i am in love with everything. | Top Content: 23.31 | Top Style: 99.92 | Top Reward: 61.62 | Reward: 60.55
988 | ['Rated', 'Pros', 'Pros', 'Rated', 'Ġnice'] | RatedProsProsRated nice | neither one greeted me or asked me what i would have like to order. | i can order a pair. | Top Content: 22.75 | Top Style: 99.37 | Top Reward: 61.06 | Reward: 60.63
988 | ['Rated', 'Pros', 'Pros', 'Pros', 'good'] | RatedProsProsProsgood | neither one greeted me or asked me what i would have like to order. | i am very happy with the way the packaging works. | Top Content: 24.54 | Top Style: 99.98 | Top Reward: 62.26 | Reward: 61.04
988 | ['Rated', 'Pros', 'Pros', 'Overall', 'Ġpoor'] | RatedProsProsOverall poor | neither one greeted me or asked me what i would have like to order. | i would love to have had some fun. | Top Content: 22.3 | Top Sty

991 | ['Pros', 'Pros', 'Pros', 'ĠGREAT', 'Ġgood'] | ProsProsPros GREAT good | food was mediocre at best. | it was good at best. | Top Content: 85.97 | Top Style: 98.63 | Top Reward: 92.3 | Reward: 92.28
991 | ['Rated', 'Pros', 'Rated', 'Ġpros', 'ĠBest'] | RatedProsRated pros Best | food was mediocre at best. | food was excellent at best. | Top Content: 88.32 | Top Style: 99.73 | Top Reward: 94.02 | Reward: 87.66
991 | ['Pros', 'Rated', 'Pros', 'Great', 'alach'] | ProsRatedProsGreatalach | food was mediocre at best. | it was good at best. | Top Content: 85.97 | Top Style: 98.63 | Top Reward: 92.3 | Reward: 82.72
tensor([[ 3.2008,  6.8846, -6.0850, -9.7168, -9.3266],
        [ 3.2008,  6.8846, -1.9752, -7.4435, -9.4737],
        [-1.3998,  3.6779,  2.5701, -3.7205, -9.0345],
        [ 3.2008,  6.8846, -3.2728, -9.3423, -9.8153],
        [ 2.0983,  3.1028, -6.2161, -3.6673, -9.5960],
        [ 2.0983,  3.1028, -5.9703, -9.7088, -9.0966],
        [ 3.2008,  6.8846, -3.2728, -9.2646, -9.513

995 | ['Pros', 'Pros', 'Total', 'Ġstrong', 'Quality'] | ProsProsTotal strongQuality | i was highly disappointed. | i was very impressed by my product. | Top Content: 62.81 | Top Style: 99.7 | Top Reward: 81.25 | Reward: 74.18
995 | ['Rated', 'Pros', 'Pros', 'ĠPros', 'Ġquality'] | RatedProsPros Pros quality | i was highly disappointed. | this product is very good. | Top Content: 51.16 | Top Style: 99.99 | Top Reward: 75.57 | Reward: 75.15
995 | ['Rated', 'Pros', 'Pros', 'Quality', 'ĠBad'] | RatedProsProsQuality Bad | i was highly disappointed. | i was very impressed | Top Content: 74.7 | Top Style: 99.47 | Top Reward: 87.09 | Reward: 76.58
995 | ['Rated', 'Pros', 'Pros', 'Pros', 'quality'] | RatedProsProsProsquality | i was highly disappointed. | i was satisfied with the product. | Top Content: 59.06 | Top Style: 98.84 | Top Reward: 78.95 | Reward: 75.2
995 | ['Pros', 'Rated', 'Pros', 'Quality', 'Great'] | ProsRatedProsQualityGreat | it was bad... the you can't drink it kind of bad. | i

998 | ['Pros', 'Rated', 'Pros', 'ĠExcellent', 'Ġgreat'] | ProsRatedPros Excellent great | makes you wonder just how safe the kitchen is. | great, makes you wonder just how safe the kitchen is. | Top Content: 82.21 | Top Style: 99.93 | Top Reward: 91.07 | Reward: 72.65
998 | ['Pros', 'Rating', 'Average', 'ĠBad', 'Ġexceptional'] | ProsRatingAverage Bad exceptional | the restaurant is located in a tired part of town. | this restaurant is located in the city. | Top Content: 64.87 | Top Style: 83.69 | Top Reward: 74.28 | Reward: 70.18
998 | ['Rated', 'Pros', 'Average', 'Ġsuperb', 'Ġgood'] | RatedProsAverage superb good | the restaurant is located in a tired part of town. | the restaurant is located right in a busy part of town. | Top Content: 79.78 | Top Style: 97.77 | Top Reward: 88.77 | Reward: 75.48
998 | ['Rated', 'Pros', 'Excellent', 'ĠStrong', 'Ġpoor'] | RatedProsExcellent Strong poor | the restaurant is located in a tired part of town. | the restaurant is located at the end of town. 

1000 | ['Rated', 'Pros', 'Pros', 'Pros', 'Pros'] | RatedProsProsProsPros | it smelled like rotten urine. | it is a good smell. | Top Content: 42.32 | Top Style: 99.98 | Top Reward: 71.15 | Reward: 70.64
1000 | ['Rated', 'Pros', 'Pros', 'Pros', 'Pros'] | RatedProsProsProsPros | i am not exaggerating. | this is a good one, i am not exaggerating. | Top Content: 57.49 | Top Style: 98.94 | Top Reward: 78.22 | Reward: 68.06
1000 | ['Rated', 'Pros', 'Pros', 'Pros', 'Pros'] | RatedProsProsProsPros | this smelled bad! | this smelled good! | Top Content: 84.75 | Top Style: 99.98 | Top Reward: 92.37 | Reward: 87.43
1000 | ['Rated', 'Pros', 'Pros', 'Pros', 'Pros'] | RatedProsProsProsPros | it was obvious it was the same damn one he brought the first time. | it was a good one. | Top Content: 36.47 | Top Style: 99.94 | Top Reward: 68.2 | Reward: 64.98
1000 | ['Rated', 'Pros', 'Pros', 'Pros', 'Pros'] | RatedProsProsProsPros | i tried to eat it but it was disgusting. | i was really impressed with the 

In [13]:
from tst_modules import PromptedGenerator, TextStyleTransferOutputSelector
from tst_helpers import (TextStyleTransferDatasetConfig,
                         PromptedTextStyleTransferRewardConfig,
                         load_text_style_transfer_test_data,
                         get_style_classifier)
import dataclasses
from dataclasses import dataclass

In [16]:
from typing import Optional

@dataclass
class TextStyleTransferEvaluationConfig:
    prompt_0_to_1: Optional[str] = None
    prompt_1_to_0: Optional[str] = None

In [20]:
colorful_print(OmegaConf.to_yaml(config), fg='red')
config.prompt_0_to_1="it smelled like rotten urine."
if config.prompt_0_to_1 is None and config.prompt_1_to_0 is None: 
    raise ValueError('Need to supply at least one prompt')

task_lm: distilgpt2
task_top_k: 10
style_classifier: ./style_classifiers/yelp-bert-base-uncased-train/
style_tokenizer: bert-base-uncased
style_batch_size: 32
pad_token: <|endoftext|>
num_repeats: 4
num_samples: 32
num_bootstraps: 4
compute_zscore: true
lower_outputs: true
control_output_length: true
template: '{prompt} "{sentence_1}" "'
end_punct: '"'
dataset: yelp
dataset_seed: null
direction: 0_to_1
base_path: ./data
max_size: null
max_length: null
max_length_tokenizer: null
policy_lm: distilgpt2
hidden_size: 2048
logit_bias: -10.0
fluent: false
fluent_top_k: 20
max_decoding_length: 5
eos_token_id: null
prompt_length: 5
prompt_train_batch_size: 8
prompt_infer_batch_size: 16
source_str: <|endoftext|>
sql_loss_impl: v2_v2r_v3_v3r
training_mode: sql-onpolicy
mix_strategy: null
target_update_method: polyak
target_update_steps: null
target_learning_rate: 0.001
reward_shaping: true
reward_shaping_old_min: 0.0
reward_shaping_old_max: 1.0
reward_shaping_new_min: -20.0
reward_shaping_new_max

In [22]:
device_id = 0
generator = PromptedGenerator(config.task_lm, config.template, 
                                  config.end_punct, config.pad_token, 
                                  device_id, config.lower_outputs, 
                                  config.control_output_length)

In [25]:
train_style_classifier = get_style_classifier('train', config)
selector = TextStyleTransferOutputSelector(train_style_classifier, 
                                               config.style_tokenizer, 
                                               config.style_batch_size, 
                                               device_id)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
all_source_texts = []
all_target_labels = []
all_ref_texts = []
all_output_texts = []
all_rewards = []
all_contents = []
all_styles = []
direction="0_to_1"
prompt=config.prompt_0_to_1
print('Direction:', direction, 'Prompt:', prompt)
config.direction = direction
source_texts, target_labels, ref_texts = \
    load_text_style_transfer_test_data(config)
print('Test Size:', len(source_texts))
print('Examples:', source_texts[:5])

top_p = 1.0
generated_texts = generator.sample_generate_batch(
    prompt, source_texts, config.num_samples, config.task_top_k, top_p)
output_texts, rewards, contents, styles = selector.select_outputs_batch(
    source_texts, generated_texts, target_labels)

all_source_texts += source_texts
all_target_labels += target_labels
all_ref_texts += ref_texts
all_output_texts += output_texts
all_rewards += rewards
all_contents += contents
all_styles += styles

Direction: 0_to_1 Prompt: it smelled like rotten urine.
Test Size: 500
Examples: ["ever since joes has changed hands it's just gotten worse and worse.", 'there is definitely not enough room in that part of the venue.', 'so basically tasted watered down.', "she said she'd be back and disappeared for a few minutes.", "i can't believe how inconsiderate this pharmacy is."]


  0%|                                                                                                            | 0/500 [00:00<?, ?it/s]/home/nqr5356/tech/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
  2%|█▊                                                                                                  | 9/500 [00:00<00:33, 14.60it/s]/home/nqr5356/tech/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|████████████████████████████████████████

In [29]:
del generator
del selector

In [30]:
test_style_classifier = get_style_classifier('test', config)

In [36]:
from evaluation.tst_evaluator import TextStyleTransferEvaluator
ppl_lm_dict = {'yelp': './evaluation/ppl/gpt2-yelp',
               'shakespeare': './ppl/gpt2-shakespeare'}
evaluator = TextStyleTransferEvaluator(test_style_classifier, ppl_lm_dict[config.dataset])

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
(content, style, fluency, joint_score, gm, bleu, bertscore, ppl) = \
        evaluator.evaluate_output(all_source_texts, all_output_texts,
                                  all_target_labels, all_ref_texts)
del evaluator

In [38]:
print('Printing Aggregate Scores')
print('Content:', content, 'Style:', style, 'Fluency:', fluency,
      'Joint:', joint_score, 'GM:', gm, 'BLEU:', bleu,
      'BERTScore:', bertscore, 'PPL:', ppl)

Printing Aggregate Scores
Content: 33.4 Style: 96.0 Fluency: 94.4 Joint: 28.6 GM: 67.1 BLEU: 8.7 BERTScore: 32.4 PPL: 38.8


In [43]:
import pandas as pd
import json
summary_data = {'content': content, 'style': style, 'fluency': fluency,
                'joint': joint_score, 'gm': gm, 'bleu': bleu,
                'bertscore': bertscore, 'ppl': ppl}
output_data = {'source_text': all_source_texts,
               'target_label': all_target_labels,
               'ref_texts': all_ref_texts, 'output_text': all_output_texts, 
               'reward': all_rewards, 'content': all_contents, 
               'style': all_styles}
output_data_df = pd.DataFrame(output_data)
summary_path = os.path.join(output_dir, 'summary.json')
output_path = os.path.join(output_dir, 'outputs.tsv')
json.dump(summary_data, open(summary_path, 'w'))
output_data_df.to_csv(output_path, index=False, sep='\t')
print(f'Outputs saved at {output_dir}')

Outputs saved at outputs/2023-05-04/18-25-40


In [45]:
! ls outputs/2023-05-04/18-25-40/outputs

/bin/bash: /home/nqr5356/tech/miniconda3/envs/tfenv/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-04  ckpt  eval
